# Préparation des données

Ce notebook présente les commandes utiles pour les principales tâches de préparation des données. Les données `notes.data` qui servent d'illustration sont des notes obtenues par des étudiants dans des matières.

In [2]:
%matplotlib inline
import pandas as pd
pd.set_option('display.max_rows', 100)     #capacité maximale d'affichage à 100 lignes

In [11]:
df = pd.read_csv("eCO2mix_RTE_En-cours-TR.csv", sep="\t", index_col=0, encoding='latin-1')
df

C:\Users\leagr\AppData\Local\Temp\ipykernel_5388\1065841346.py:1: DtypeWarning: Columns (5,26,27,30,31,33,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("eCO2mix_RTE_En-cours-TR.csv", sep="\t", index_col=0, encoding='latin-1')


,Périmètre,Nature,Date,Heures,Consommation,Prévision J-1,Prévision J,Fioul,Charbon,Gaz,...,Hydraulique - Fil de l?eau + éclusée,Hydraulique - Lacs,Hydraulique - STEP turbinage,Bioénergies - Déchets,Bioénergies - Biomasse,Bioénergies - Biogaz,Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore
France,Données temps réel,2022-06-01,00:00,44940.0,44800,45100.0,144.0,0.0,3596.0,30207.0,...,1054,1677.0,170,584.0,286.0,ND,ND,ND,ND,NaN
France,Données temps réel,2022-06-01,00:15,43967.0,43700,43900.0,144.0,0.0,3716.0,30337.0,...,1419,581.0,171,560.0,276.0,ND,ND,ND,ND,NaN
France,Données temps réel,2022-06-01,00:30,42514.0,42600,42700.0,142.0,0.0,2880.0,29746.0,...,1280,530.0,169,561.0,276.0,ND,ND,ND,ND,NaN
France,Données temps réel,2022-06-01,00:45,41073.0,41450,41600.0,143.0,0.0,2699.0,29230.0,...,1120,361.0,170,563.0,276.0,ND,ND,ND,ND,NaN
France,Données temps réel,2022-06-01,01:00,40359.0,40300,40500.0,144.0,3.0,2718.0,29019.0,...,1101,373.0,170,563.0,276.0,ND,ND,ND,ND,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
France,Données temps réel,2023-12-01,23:00,NaN,ND,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
France,Données temps réel,2023-12-01,23:15,NaN,ND,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
France,Données temps réel,2023-12-01,23:30,NaN,ND,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
France,Données temps réel,2023-12-01,23:45,NaN,ND,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df.shape

df.dtypes

Périmètre                                object
Nature                                   object
Date                                     object
Heures                                  float64
Consommation                             object
Prévision J-1                           float64
Prévision J                             float64
Fioul                                   float64
Charbon                                 float64
Gaz                                     float64
Nucléaire                               float64
Eolien                                  float64
Solaire                                 float64
Hydraulique                             float64
Pompage                                 float64
Bioénergies                             float64
Ech. physiques                          float64
Taux de Co2                             float64
Ech. comm. Angleterre                   float64
Ech. comm. Espagne                      float64
Ech. comm. Italie                       

In [14]:
df.count()

Périmètre                               52704
Nature                                  52704
Date                                    52704
Heures                                  52479
Consommation                            52704
Prévision J-1                           52512
Prévision J                             52479
Fioul                                   52479
Charbon                                 52479
Gaz                                     52479
Nucléaire                               52479
Eolien                                  52479
Solaire                                 52479
Hydraulique                             52479
Pompage                                 52479
Bioénergies                             52479
Ech. physiques                          52479
Taux de Co2                             26256
Ech. comm. Angleterre                   26256
Ech. comm. Espagne                      26256
Ech. comm. Italie                       26256
Ech. comm. Suisse                 

In [13]:
df.nunique()    # surtout utile pour les variables catégoriques

Périmètre                                   1
Nature                                    549
Date                                       96
Heures                                  26699
Consommation                             1973
Prévision J-1                            2035
Prévision J                              1147
Fioul                                    1594
Charbon                                  8782
Gaz                                     20068
Nucléaire                               12250
Eolien                                  10009
Solaire                                  9844
Hydraulique                              3592
Pompage                                   654
Bioénergies                             21093
Ech. physiques                            110
Taux de Co2                              5255
Ech. comm. Angleterre                    3032
Ech. comm. Espagne                       3619
Ech. comm. Italie                        4128
Ech. comm. Suisse                 

<br>

## 1. Sélectionner les enregistrements

### Sélection

Utilisez la fonction `query()` pour filtrer les enregistrements selon un critère booléen

In [30]:
charbon = df["Charbon"]
charbon.head()

France    3596.0
France    3716.0
France    2880.0
France    2699.0
France    2718.0
Name: Charbon, dtype: float64

In [27]:
df.query('Charbon > 16')

,Périmètre,Nature,Date,Heures,Consommation,Prévision J-1,Prévision J,Fioul,Charbon,Gaz,...,Hydraulique - Fil de l?eau + éclusée,Hydraulique - Lacs,Hydraulique - STEP turbinage,Bioénergies - Déchets,Bioénergies - Biomasse,Bioénergies - Biogaz,Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore
France,Données temps réel,2022-06-01,00:00,44940.0,44800,45100.0,144.0,0.0,3596.0,30207.0,...,1054,1677.0,170,584.0,286.0,ND,ND,ND,ND,NaN
France,Données temps réel,2022-06-01,00:15,43967.0,43700,43900.0,144.0,0.0,3716.0,30337.0,...,1419,581.0,171,560.0,276.0,ND,ND,ND,ND,NaN
France,Données temps réel,2022-06-01,00:30,42514.0,42600,42700.0,142.0,0.0,2880.0,29746.0,...,1280,530.0,169,561.0,276.0,ND,ND,ND,ND,NaN
France,Données temps réel,2022-06-01,00:45,41073.0,41450,41600.0,143.0,0.0,2699.0,29230.0,...,1120,361.0,170,563.0,276.0,ND,ND,ND,ND,NaN
France,Données temps réel,2022-06-01,01:00,40359.0,40300,40500.0,144.0,3.0,2718.0,29019.0,...,1101,373.0,170,563.0,276.0,ND,ND,ND,ND,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
France,Données temps réel,2023-11-29,14:30,64161.0,65900,64400.0,179.0,1.0,7819.0,43250.0,...,3444.0,1261.0,76.0,515.0,246.0,-5.0,0.0,1255.0,118.0,NaN
France,Données temps réel,2023-11-29,14:45,64239.0,65600,64100.0,179.0,1.0,7905.0,43255.0,...,3726.0,1580.0,73.0,519.0,246.0,-3.0,11.0,1295.0,125.0,NaN
France,Données temps réel,2023-11-29,15:00,64370.0,65300,63800.0,178.0,2.0,7914.0,43253.0,...,3667.0,1903.0,76.0,507.0,246.0,-5.0,13.0,1274.0,121.0,NaN
France,Données temps réel,2023-11-29,15:15,64097.0,65100,63600.0,178.0,100.0,8281.0,43179.0,...,3535.0,1763.0,75.0,506.0,246.0,-4.0,3.0,1254.0,133.0,NaN


Comme la fonction renvoie un DataFrame, vous pouvez enchaîner plusieurs filtres

In [ ]:
df.query('Logiques > 16').query('Stats > 16')

On peut aussi créer un critère plus complexe à l'aide des opérateurs `and` et `or`

In [ ]:
df.query('Logiques > 16 and Stats > 16')

In [ ]:
df.query('Logiques > 16 or (Stats > 16 and Systèmes == "B")')

Notez les double quotes " dans la chaîne délimitée par les simples quotes '

Pour une variable catégorique, on peut avoir besoin de filtrer sur plusieurs modalités possibles :

In [ ]:
df.query('Systèmes in ["C", "D"]')

Il est également possible d'appeler une fonction dans la chaîne de caractères. Par exemple pour isoler les lignes qui contiennent des valeurs manquantes dans une colonne, on peut utiliser les méthodes `isna()` ou `isnull()` :

In [ ]:
df.query('Logiques.isna()')

### Suppression

Pour supprimer définitivement une ou plusieurs ligne, on peut réaffecter le DataFrame :

In [ ]:
df2=df.query('Systèmes in ["C", "D"]')
df2

Pour supprimer selon les noms de lignes, on peut utiliser la fonction `drop()`.

In [ ]:
df2=df.drop('Carine TRUDELLE')
df2.shape

In [ ]:
df2=df.drop(['Ambroise LEGUENNEC','Roméo LEBERGUEILLEC'])
df2.shape

Notez que dans la plupart des DataFrames, le nom des lignes est un numéro de lignes. Dans ce cas, une commande du type `df.drop(0)` par exemple permet de supprimer la première ligne (indice 0).

<br>

## 2. Sélectionner et créer les variables

### Sélection

Revoir le notebook Introduction.ipynb

In [ ]:
df.Sport

In [ ]:
df['Sport']

In [ ]:
df.iloc[:,2:5]

### Suppression

In [ ]:
df2 = df.drop('Sport', axis=1)    # axis=1 indique qu'on agit sur les colonnes, la fonction agit sur les lignes par défaut
df2

### Création d'une nouvelle variable

Il faut utiliser la syntaxe avec des crochets. Les méthodes `loc` ou `iloc` ne modifieront pas le DataFrame.

In [ ]:
df['Moyenne des notes'] = df.mean(axis=1)
df.head()

In [ ]:
df['CopieSport'] = df.Sport
df['Nom'] = df.index    # Recopier l'Index dans une colonne
df.head()

### Renommage des variables

In [ ]:
df.columns = ['EPS','BD','LogiquesClassiques','Statistiques','SI', 'Moyenne', 'CopieEPS', 'Etudiant']
df.head()

Il est également possible de renommer l'index.

In [ ]:
df.index.name

In [ ]:
df.index.name = 'Prénom_Nom'
df.head()

Pour la suite du notebook, on réinitialise le DataFrame.

In [ ]:
df = pd.read_csv("notes.data", sep="\t", index_col=0)

<br>

## 3. Vérifier et transformer les types des variables

In [ ]:
df.dtypes

Les types des variables ont été détectés automatiquement d'après les données. Si les types ne conviennent pas, **il faut les changer !**

### Mettre une colonne en type numérique

Transformons la variable catégorique `Systèmes` en une variable numérique. Les valeurs A, B, C et D doivent être remplacées par les notes 18, 14, 10 et 6.

In [ ]:
df.Systèmes = df.Systèmes.replace(to_replace = 'A', value = '18')
df.Systèmes = df.Systèmes.replace(to_replace = 'B', value = '14')
df.Systèmes = df.Systèmes.replace(to_replace = 'C', value = '10')
df.Systèmes = df.Systèmes.replace(to_replace = 'D', value = '6')

Il est possible de remplacer dans tout le DataFrame, sans cibler une colonne, avec `df.replace(to_replace = 'A', value = '18')`

On peut aussi faire le remplacement avec un seul appel à la fonction :

In [ ]:
df.replace(to_replace =  {'A': '18', 'B': '14',  'C': '10', 'D': '6'})

In [ ]:
df.dtypes

Il reste à changer le type de la colonne `Systèmes` pour la rendre numérique. On utilise la fonction `to_numeric()` :

In [ ]:
df.Systèmes = pd.to_numeric(df.Systèmes)
df.dtypes

In [ ]:
df.head()

### Nettoyer une colonne pour la rendre numérique

Des problèmes récurrents empêchent les variables numériques d'être stockées dans un type numérique :
- la présence d'espaces pour séparer les chiffres, comme dans `2 154 120`
- la présence d'une virgule à la place d'un point, comme dans `127,79`
- la présence d'espaces ou de caractères invisibles au début ou à la fin de la chaîne
- la présence dans la colonne de valeurs manquantes codées avec des caractères, comme `?`
- etc.

Pour remplacer des caractères dans une colonne du DataFrame, utilisez la fonction `str.replace()`. Ci-dessous un exemple avec un DataFrame à une seule colonne.

In [ ]:
dfsale = pd.DataFrame({"montant": ["26 460,15", "29 842,70", "29 074,20"]})
dfsale

In [ ]:
dfsale.dtypes

In [ ]:
dfsale.montant = pd.to_numeric(dfsale.montant)   # renvoie une erreur

In [ ]:
dfsale.montant = dfsale.montant.str.replace(",",".")
dfsale.montant = dfsale.montant.str.replace(" ","")
dfsale.montant = pd.to_numeric(dfsale.montant)
dfsale

In [ ]:
dfsale.dtypes

Pour supprimer les espaces, tabulations et caractères invisibles au début et à la fin des chaînes, appliquez la fonction `str.strip()` sur la colonne.

### Mettre une colonne en type chaîne de caractères (string)

Si une colonne numérique contient en fait des catégories (par exemple les valeurs 1, 2 et 3 pour indiquer un numéro de trimestre), il vaut mieux traiter la colonne comme une variable catégorique pour éviter les erreurs dans l'analyse. La commande ci-dessous permet de transformer une colonne numérique vers le type chaîne de caractères.

In [ ]:
df.Sport = df.Sport.astype(str)
df.dtypes

Plutôt que des chaînes de caractères (str), il existe en pandas un véritable type pour les variables catégoriques :

In [ ]:
df.Stats = df.Stats.astype('category')
df.dtypes

Son principal intérêt est de permettre de déclarer un ordre entre les catégories qui ne correpond pas à l'ordre alphabétique/alphanumérique. Ce format prend également moins de place en mémoire.

Pour la suite du notebook, on réinitialise le DataFrame.

In [ ]:
df = pd.read_csv("notes.data", sep="\t", index_col=0)

### Mettre une colonne en type date

Si une colonne du DataFrame contient des dates, elles sont par défaut reconnues comme des chaînes de caractères. Il est possible de transformer la colonne en type date à l'aide de cette commande :

In [ ]:
# df.date = pd.to_datetime(df.date)
# commande mise en commentaire pour la bonne exécution du notebook

<br>

## 4. Agréger des lignes

La fonction `groupby()` permet de **regrouper** les lignes selon une ou plusieurs colonnes. Le but final est d'**agréger** les valeurs par groupe dans une ou plusieurs autres colonnes.

In [ ]:
df.groupby('Systèmes')

L'objet retourné n'est pas affichable nativement.

Pour agréger, pandas propose des fonctions d'agrégation, dont la plus basique est `size()` :

In [ ]:
df.groupby('Systèmes').size()    # nombre de lignes dans chaque groupe

La fonction `size()` renvoie toujours une Series. Notez que les groupes se retrouvent en Index de la Series, tandis que les valeurs agrégées remplissent la Series, comme on peut le voir ci-dessous :

In [ ]:
agregation = df.groupby('Systèmes').size()
agregation.index

In [ ]:
agregation.values

Si on regroupe selon plusieurs colonnes à la fois, on obtient une Series avec un index composite (à plusieurs colonnes), nommé MultiIndex.

In [ ]:
agregation = df.groupby(['Systèmes','Sport']).size()
agregation

In [ ]:
agregation.index

Si la fonction `size()` renvoie toujours une Series, les autres fonctions d'agrégation renvoient :
- une Series si on n'agrège que dans une seule colonne,
- un DataFrame sinon.

Par exemple avec la fonction `mean()` qui permet d'agréger en calculant la moyenne, cela donne :

In [ ]:
df.groupby('Systèmes').mean()    # Moyenne calculée dans chaque colonne numérique

In [ ]:
df.groupby('Systèmes').Logiques.mean()    # Moyenne calculée dans la colonne Sport uniquement 

Il est aussi possible de grouper avec le paramètre `as_index=False`. Dans ce cas, les groupes ne sont plus stockés dans un Index ou MultiIndex mais directement dans des colonnes. Le résultat est forcément un DataFrame.

In [ ]:
df.groupby('Systèmes',as_index=False).mean()    # Grouper sur 1 colonne, agréger dans toutes les colonnes numériques

In [ ]:
df.groupby('Systèmes',as_index=False).Logiques.mean()    # Grouper sur 1 colonne, agréger dans 1 colonne

In [ ]:
df.groupby(['Systèmes','Sport'],as_index=False).mean()    # Grouper sur 2 colonnes, agréger dans toutes les colonnes numériques

In [ ]:
df.groupby(['Systèmes','Sport'],as_index=False).Logiques.mean()    # Grouper sur 2 colonnes, agréger dans 1 colonne

Remarques :

Le paramètre `as_index=False` du `groupby()` ne fonctionne pas si l'agrégation est réalisée avec la fonction `size()`. On obtient quand même une Series avec les groupes en index.

Après une agrégation avec `size()`, pour transformer la Series en DataFrame, on peut utiliser une de ces deux solutions :

In [ ]:
# Utilisation de la fonction to_frame()
df.groupby('Systèmes').size().to_frame('Effectif')

In [ ]:
# Utilisation du constructeur de la classe DataFrame
pd.DataFrame(df.groupby('Systèmes').size(), columns=['Effectif'])

En plus de `size()` et `mean()`, les principales fonctions d'agrégation sont :

In [ ]:
df.groupby('Systèmes').Sport.sum()    # somme

In [ ]:
df.groupby('Systèmes').Sport.min()    # minimum

In [ ]:
df.groupby('Systèmes').Sport.max()    # maximum

In [ ]:
df.groupby('Systèmes').Sport.count()    # nombre de valeurs dans la colonne, mais sans les valeurs manquantes, contrairement à size()

In [ ]:
df.groupby('Systèmes').Sport.nunique()    # nombre de valeurs distinctes dans la colonne

<br>

## 5. Transformer des variables

A venir...

<br>

## 6. Gérer les valeurs manquantes

### Repérer les valeurs manquantes

On peut compter les lignes ou les colonnes qui contiennent des valeurs manquantes.

In [ ]:
df.isna().sum() 

In [ ]:
df.isna().sum(axis=1).sort_values(ascending=False)

Pour isoler les lignes qui contiennent une valeur manquante, utilisez :

In [ ]:
df[df.isna().any(axis=1)]

On peut cibler uniquement les valeurs manquantes d'une colonne uniquement, par exemple avec la commande ci-dessous.

In [ ]:
df.query('Systèmes.isna()')

### Imputer les valeurs manquantes

On peut remplacer les valeurs manquantes en utilisant la fonction `fillna()`.

Pour imputer par la moyenne dans une colonne :

In [ ]:
df.Stats.fillna(df.Stats.mean(), inplace=True)

In [ ]:
df.isna().sum() 

Pour réaliser des imputations plus précises, on peut appliquer une technique de clustering sur les données puis imputer par la moyenne ou le mode dans chaque cluster. Une autre solution consiste à réaliser un modèle supervisé pour "prédire" la valeur manquante en fonction des autres variables.

### Supprimer les lignes/colonnes

En dernier recours, la fonction `dropna()` permet de supprimer les lignes ou colonnes qui contiennent au moins une valeur manquante. Il est possible d'exiger un seuil supérieur de valeurs manquantes pour que la ligne/colonne soit supprimée (voir la [doc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)).

De manière générale, on déconseille de supprimer des lignes si cela représente plus de 5% du jeu de données.

In [ ]:
df.dropna()

In [ ]:
df.dropna(axis=1)

N'oubliez pas de réaffecter le résultat dans une variable, ou d'utiliser le paramètre `inplace=True`.

Pour une vue plus complète sur la gestion des valeurs manquantes, voir la doc de [pandas](http://pandas.pydata.org/) : https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html